# Python과 DataBase 연동

1. 해당 라이브러리 : DB 접속 드라이버, cx_Oracle, python-oracledb
  - 설치 : Anaconda prompt 창(관리자 권한 실행) pip install 설치할 모듈명
  - 주피터 노트북안에서 설치: !pip install 설치할 모듈명
  - Anaconda : pip install or conda install
  - pip install oracledb -- upgrade 최근 버전(권장)
  - pip install cx_Oracle -- upgrade
  
2. 접속정보 확인
  - 사용자 계정과 비밀번호 : scott/tiger
  - 오라클 url : localhost:1521/xe

# DB 연동 작업


In [1]:
!pip install cx_Oracle --upgrade

In [6]:
# 라이브러리(모듈) 연결
import cx_Oracle as cx

In [7]:
# db 연동 connect('사용자계정', '비밀번호', 'url')
# url - localhost(127.0.0.1):1521 
conn = cx.connect('scott','1234','localhost:1521/xe')

## DB에 테이블 조회

In [4]:
#cursor(커서) : 레코드 셋을 탐색하는 포인터
#커서 객체 생성
cur = conn.cursor()

sql = 'select * from product'

cur.execute(sql)

for row in cur:
    print(row)

(1, '레몬', 1500, <cx_Oracle.LOB object at 0x000001B667868840>, 'lemon.jpg')
(2, '오렌지', 1500, <cx_Oracle.LOB object at 0x000001B6678635D0>, 'orange.jpg')
(3, '키위', 1500, <cx_Oracle.LOB object at 0x000001B66784DF00>, 'kiwi.jpg')
(4, '포도', 1500, <cx_Oracle.LOB object at 0x000001B667875B70>, 'grape.jpg')
(5, '딸기', 1500, <cx_Oracle.LOB object at 0x000001B667875C30>, 'strawberry.jpg')


## 전체 레코드 삭제후 테이블에 레코드 삽입하는 작업

In [5]:
sql = 'delete from product'
cur.execute(sql)
conn.commit()

In [6]:
#파이썬에서 DB에 레코드 삽입

items = [(1, '레몬', 1500, '맛있는 레몬', 'lemon.jpg'),
(2, '오렌지', 1500, '맛있는 오렌지', 'orange.jpg'),
(3, '키위', 1500, '맛있는 키위', 'kiwi.jpg'),
(4, '포도', 1500, '맛있는 포도', 'grape.jpg'),
(5, '딸기', 1500, '맛있는 딸기', 'strawberry.jpg')]


for row in items:
    sql = 'insert into product values (:1, :2, :3, :4, :5)'
    print(row)
    cur.execute(sql, row)
    

conn.commit()

(1, '레몬', 1500, '맛있는 레몬', 'lemon.jpg')
(2, '오렌지', 1500, '맛있는 오렌지', 'orange.jpg')
(3, '키위', 1500, '맛있는 키위', 'kiwi.jpg')
(4, '포도', 1500, '맛있는 포도', 'grape.jpg')
(5, '딸기', 1500, '맛있는 딸기', 'strawberry.jpg')


In [7]:
sql = 'select count(*) from product'
cur.execute(sql)

count = cur.fetchone()
print('상품갯수 : ', count)


상품갯수 :  (5,)


In [8]:
sql = 'delete from product'
cur.execute(sql)
conn.commit()

In [9]:
items = [(1, '레몬', 1500, '맛있는 레몬', 'lemon.jpg'),
(2, '오렌지', 1500, '맛있는 오렌지', 'orange.jpg'),
(3, '키위', 1500, '맛있는 키위', 'kiwi.jpg'),
(4, '포도', 1500, '맛있는 포도', 'grape.jpg'),
(5, '딸기', 1500, '맛있는 딸기', 'strawberry.jpg')]

sql = 'insert into product values(:1, :2, :3, :4, :5)'
cur.bindarraysize = len(items)
cur.executemany(sql, items)
conn.commit()


In [10]:
cur.close()
conn.close()

# 회원관리 프로그램

- CRUD(CREATE, SELECT, UPDATE, INSERT, DELETE) 즉, DML(조작어)
- 테이블생성 Member(userid, userpwd, name), userid는 primary key 설정
- 주요기능 : 회원가입(등록), 회원수정, 회원정보출력, 회원삭제 등



## 레코드 수정

In [11]:
def update(userid, userpass, username):
    conn = cx.connect('scott', '1234', 'localhost:1521/xe')
    sql = 'update member set userpass=:1 username=:2 where userid=:3'
    
    cur = conn.cursor()
    cur.execute(sql, [userpass, username, userid])
    
    cur.close()
    conn.commit()
    conn.close()

In [5]:
def id_check(userid):
    conn = cx.connect('scott', '1234', 'localhost:1521/xe')
    sql = 'select * from member'
    
    cur = conn.cursor()
    cur.execute(sql)
    rs = cur.fetchall()
    
    for row in rs:
        if row[0] == userid:
            return True
    
    return False

In [ ]:
# 키보드를 통해 회원정보를 변경하는 작업
userid = input('id = ')
if id_check(userid):
    userpass = input('pass = ')
    username = input('name = ')
    update(userid, userpass, username)
else:
    print(f'{userid} 회원정보가 존재하지 않습니다.')

## main processor

In [9]:
from dbTest import *

while True:
    print('\n== 회원관리 프로그램 ==')
    print('1. 회원등록')
    print('2. 회원수정')
    print('3. 회원삭제')
    print('4. 회원출력')
    print('5. 작업종료')
    menu = int(input('메뉴 = '))
    
    if menu == 1:
        userid = input('id = ')
        userpass = input('pass = ')
        username = input('name = ')
        insert(userid, userpass, username)
        
    elif menu == 2:
        userid = input('id = ')
        if id_check(userid) == 0:
            print(f'{userid} 회원정보가 존재하지 않습니다.')
        else:
            print('수정할 정보를 입력하세요')
            userpass = input('pass = ')
            username = input('name = ')
            update(userid, userpass, username)
            
    elif menu == 3:
        userid = input('id = ')
        if id_check(userid) == 0:
            print(f'{userid} 회원정보가 존재하지 않습니다.')
        else:
            delete(userid)
    elif menu == 4:
        listMember()
    elif menu == 5:
        close()
        break
    else:
        print('메뉴의 선택이 올바르지 않습니다.')

print('프로그램을 종료합니다.')


== 회원관리 프로그램 ==
1. 회원등록
2. 회원수정
3. 회원삭제
4. 회원출력
5. 작업종료


KeyboardInterrupt: Interrupted by user

In [11]:
from project2 import *

while True:
    print('\n== 학생관리 프로그램 ==')
    print('1. 신규 학생 등록')
    print('2. 학생 정보 수정')
    print('3. 학생 점수 수정')
    print('4. 학생 정보 삭제')
    print('5. 학생 정보 조회')
    print('6. 전체 학생 조회')
    print('7. 프로그램 종료')
    menu = int(input('메뉴 = '))
    
    if menu == 1:
        name = input('이름 = ')
        num = input('학번 = ')
        sw = True
        while sw:
            gender = input('성별(F/M) = ')
            if gender != 'F' and gender != 'M':
                sw = True
            else:
                sw = False
        kor = input('국어 = ')
        eng = input('영어 = ')
        math = input('수학 = ')
        total = kor + eng + math
        insert(name, num, gender, kor, eng, math)
        
    elif menu == 2:
        num = input('학번 = ')
        if id_check(num) == 0:
            print(f'{userid} 학생정보가 존재하지 않습니다.')
        else:
            print('수정할 정보를 입력하세요')
            name = input('이름 = ')
            num = input('학번 = ')
            sw = True
            while sw:
                gender = input('성별(F/M) = ')
                if gender != 'F' and gender != 'M':
                    sw = True
                else:
                    sw = False
            update_n(name, num, gender)
          
    elif menu == 3:
        num = input('학번 = ')
        if id_check(num) == 0:
            print(f'{num} 학생정보가 존재하지 않습니다.')
        else:
            print('수정할 점수를 입력하세요')
            kor = input('국어 = ')
            eng = input('영어 = ')
            math = input('수학 = ')
            update_s(kor, eng, math)
            
    elif menu == 4:
        num = input('학번 =')
        if id_check(num) == 0:
            print(f'{num} 학생정보가 존재하지 않습니다.')
        else:
            delete(num)
        
    elif menu == 5:
        num = input('학번 = ')
        if id_check(num) == 0:
            print(f'{num} 학생정보가 존재하지 않습니다.')
        else:
            member(num)
        
    elif menu == 6:
        listMember()
    elif menu == 7:
        print('프로그램을 종료합니다.')
        close()
        break
    
    else:
        print('메뉴의 선택이 올바르지 않습니다.')



== 학생관리 프로그램 ==
1. 신규 학생 등록
2. 학생 정보 수정
3. 학생 점수 수정
4. 학생 정보 삭제
5. 학생 정보 조회
6. 전체 학생 조회
7. 프로그램 종료
메뉴 = 1
이름 = a
학번 = 1
성별(F/M) = F
국어 = 43
영어 = 56
수학 = 76


DatabaseError: ORA-00913: too many values